In [1]:
import pandas as pd
import requests
import time
from xml.etree import ElementTree
import sys

In [2]:
game_ratings = pd.DataFrame([])
user_games = pd.DataFrame([])

In [3]:
def addgame(df, game_id):
    if not(df.empty):
        if df.ix[[game_id]].T.sum().iloc[0] > 0:
            print('Skipping %d, already loaded.' % game_id)
            return
    
    url = "http://www.boardgamegeek.com/xmlapi/boardgame/%d?comments=1&pagesize=100" % game_id
    page = 1
    
    total = 0
    votes = 0

    try:
        while(True):
            sys.stdout.write('.')
            sys.stdout.flush()
    #         print('Getting page %d...' % page)
            r = requests.get('%s&page=%d' % (url,page))
            tree = ElementTree.fromstring(r.content)
            assert tree.tag == 'boardgames'

            if page == 1:
    #             print(r.content)
                print('%d is %s' % (game_id, tree.findall(".//name[@primary='true']")[0].text))

            comments = tree.findall(".//comment")
    #         print("comments ", len(comments))
    #         print()

            for comment in comments:
                if comment.get('rating') != 'N/A':
    #                 print(comment.get('username'),comment.get('rating'))
                    df.loc[game_id,comment.get('username')] = float(comment.get('rating'))
                    votes = votes + 1
                    total = total + float(comment.get('rating'))

            if len(comments) != 100:
                break

            page = page + 1

        print('%d votes added.' % total)
    except:
        print('Caught an error... :(')
        pass
    
    return df

# addgame(game_ratings, 205637)

In [4]:
def get_user(user_games, user):
    print('Getting user %s' % user)
    url = "http://www.boardgamegeek.com/xmlapi/collection/" + user
    r = requests.get(url)
    while(r.status_code == 202):
        time.sleep(1)
        print("Slept 1")
        r = requests.get(url)
        
#     print(r.content)
    tree = ElementTree.fromstring(r.content)
    
    for game in tree.findall(".//item[@subtype='boardgame']"):
        user_games.loc[user, game.get('objectid')] = 1
        
#     print(r.content)
    
    return user_games

def find_a_user(game_ratings, user_games):
    user = list(set(game_ratings.columns.values).difference(user_games.T.columns.values))[0]
    return get_user(user_games, user)

def find_game(game_ratings, user_games):
    for game_id in user_games.sum().sort_values(ascending=False).index.values:
        game_id = int(game_id)
        got = False
        if not(game_ratings.empty):
            if game_ratings.ix[[game_id]].T.sum().iloc[0] > 0:
#                 print('Got %d already' % game_id)
                got = True
                
        if not(got):
            print('Found %d' % game_id)
        
            return addgame(game_ratings, game_id)

    print('Getting more users')
    find_a_user(game_ratings, user_games)
    return find_game(game_ratings, user_games)
        
# find_a_user(game_ratings, user_games)


In [ ]:
# import numpy as np
# for i in np.arange(100):
#     find_a_user(game_ratings, user_games)

# get_user(user_games, 'hitechbunny')
while(True):
    print(find_game(game_ratings, user_games).shape)
    if game_ratings.index.values.shape[0] % 50 == 0:
        game_ratings.to_csv('game_ratings.csv', encoding='utf-8')

Found 39707
.39707 is Dominion: Envoy Promo Card
........3785 votes added.
(632, 55928)
Found 21050
.21050 is Combat Commander: Europe
...............10674 votes added.
(633, 55992)
Found 22825
.22825 is Tide of Iron
...........6043 votes added.
(634, 56017)
Found 146886
.146886 is La Granja
........5112 votes added.
(635, 56027)
Found 73171
.73171 is Earth Reborn
.........5180 votes added.
(636, 56054)
Found 66589
.66589 is Navegador
.............8276 votes added.
(637, 56057)
Found 137269
.137269 is Spyrium
......

In [ ]:
# print game_ratings.shape
# (game_ratings.notnull().sum() < 2).index.values.shape
# game_ratings.notnull().sum() < 2
# game_ratings.columns.values[ (game_ratings.notnull().sum() < 2).values ]
game_ratings.drop(game_ratings.columns.values[ (game_ratings.notnull().sum() < 4).values ], axis=1).shape
# compact = game_ratings.drop((game_ratings.notnull().sum() < 2).index.values, axis=1)
# compact

In [14]:
import math

game_ratings.index.values.shape[0] % 10



9

NameError: name 'np' is not defined

In [ ]:
user_games

In [ ]:
df.ix[[123]].T.sum().iloc[0]